Takes downloaded Course Activity csv file and prints out custom emails to let students know what they need to be reading.

In [1]:
import pandas as pd
import csv

**Note:** Replace the values list in `should_have_read` with the appropriate resources for your assignment.

In [2]:
def needs_read(students_name):
    should_have_read = ['Two principles of ethics', 'What is an ethical dilemma?', '② Reading: What is a dilemma?',
                   'Ethical Dilemma or Not?', 'What is Ethics? (guest lecture)', '① Lecture', 
                   'Nature of Ethics self-check debriefing', '② Outcomes', 'Draft Ethical Dilemmas',
                   'Meaning of "principle"']
    
    
    student_read = df[df.Student_Name == students_name].Content.drop_duplicates().tolist()
    
    need_to_read = [x for x in should_have_read if x not in student_read]
    return(need_to_read)

In [3]:
activity_report_path = '/Users/briandomino/Downloads/activity 9-5-2021.csv'

In [4]:
df = pd.read_csv(activity_report_path)

This cleaning step is not strickly necessary, but I find it helpful if I need to look at the dataframe from within the notebook, e.g., to copy the resource names to paste into `should_have_read` above.

In [5]:
# Remove unneeded columns
df = df.drop(['Section Id', 'Section Name', 'Course Id', 'Course Name', 'Content Type',
                     'Times Viewed', 'Times Participated', 'Start Date', 'Student Id'], axis=1)

In [6]:
# Rename columns so we can use their names more easily
df = df.rename(columns={"Student Name": "Student_Name", 'Sortable Name': 'Sortable_Name',
                   'Student SIS ID': 'Student_ID', 'Content Name': 'Content', 'First Viewed': 'First_Viewed',
                  'Last Viewed': 'Last_Viewed'})

The `activity.csv` file will list every resource each student has looked at. I like to remove the more common irrelevant ones such as images. Again this isn't strickly necessary; it just makes life easier if you need to interact with the dataframe manually. You will need to change these values to suit your course.

In [1]:
# Content it doesn't matter whether they've looked at
stops = ['Course Home', 'Course People', 'headshot.jpg', 'ethics v legal.jpg', 'MoralityWordCloud.jpg',
         'moral dilemma ahead.jpeg', 'Course Grades', 'Course Modules', 'Google Apps']

In [8]:
# Remove stops
for i in range(0, len(stops)):
    df = df.loc[~((df['Content'] == stops[i])),:]

In [9]:
#remove rows where student looked at page < 1 second
df = df[df['First_Viewed']!=df['Last_Viewed']]

In [10]:
name_list = df['Student_Name'].unique().tolist()

In [11]:
closing_message = """Also note that this is intended to be helpful and is neither a criticism nor a graded 
activity. If you believe that the information is incorrect, I would encourage you to double-check that you
have done all of the readings in Module ②. And don't worry; your grade was not lowered."""

In [12]:
closing_message = closing_message.replace('\n', '')

The cell below will print the emails to the screen. I do this as a dry run. The output will look like this:

>Email to: studentid
>
>Subject = Readings to do before the next assignment
>Dear Jane,
>
>According to Canvas you have not yet done the following readings, which are necessary for the upcoming assignment: Two principles of ethics', 'What is an ethical dilemma?', '② Reading: What is a dilemma?'.
>
>
>Please note: due to a limitation in Canvas, this information may be as much as 40 hours old.
>This email is intended to be helpful and is neither a criticism nor a graded activity. If you believe that the information is incorrect, I would encourage you to double-check that you have done all of the readings in Module ②. And don't worry; your grade was not lowered.

In [ ]:
for name in name_list:
    needs_to_read = needs_read(name)
    if len(needs_to_read) > 5:
        print('HIGH HIGH HIGH')
    print(f'Email to: {df[df.Student_Name == name].Student_ID.unique()}\n')
    print('Subject = Readings to do before the next assignment')
    print(f'Dear {name.split()[0]},\n')
    print(f'According to Canvas you have not yet done the following readings, which are necessary for the upcoming assignment: {", ".join(map(str, needs_to_read))}.')
    print('\nPlease note: due to a limitation in Canvas, this information may be as much as 40 hours old.')
    print(closing_message + '\n')
    print('-----')

This creates the CSV file that you will upload to Google Sheets to run the mail merge.

In [13]:
with open('dilemma_need_to_read.csv', 'w') as csvfile:
        fieldnames = ['address', 'subject', 'message']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        
        for name in name_list:
            
            needs_to_read = needs_read(name)
            email_address = df[df.Student_Name == name].Student_ID.unique()
            email_address = str(email_address).replace('[', '').replace(']', '').replace('\'','') + '@miamioh.edu'
            
            subject = 'Readings to do before the next assignment'
            message = f'Dear {name.split()[0]},\nAccording to Canvas you have not yet done the following readings, which are necessary for the upcoming assignment: {", ".join(map(str, needs_to_read))}.'
            message = message + '\n\nPlease note: due to a limitation in Canvas, this information may be as much as 40 hours old. '
            message = message + closing_message + '\n'
            
            
            writer.writerow({'address': email_address, 'subject': subject, 'message': message})
